# RETAIL DEMAND PREDICTION - BASELINE MODEL

In [1]:
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import mean_squared_log_error as msle
from sklearn.model_selection import train_test_split, KFold

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

## 1. Loading datasets and merging tables

In [2]:
train_df = pd.read_csv('updated_train_data.csv')
product_df = pd.read_csv('updated_product_data.csv')
store_df = pd.read_csv('updated_store_data.csv')

In [3]:
train_df.head(2)

,WEEK_END_DATE,STORE_NUM,UPC,BASE_PRICE,FEATURE,DISPLAY,UNITS,MONTH
0,2009-01-14,367,1111009477,1.57,0,0,13,1
1,2009-01-14,367,1111009497,1.39,0,0,20,1


In [4]:
product_df.head(2)

,UPC,MANUFACTURER_1,MANUFACTURER_2,MANUFACTURER_3,MANUFACTURER_4,MANUFACTURER_5,MANUFACTURER_6,MANUFACTURER_7,MANUFACTURER_8,MANUFACTURER_9,...,CATEGORY_3,CATEGORY_4,SUB_CATEGORY_1,SUB_CATEGORY_2,SUB_CATEGORY_3,SUB_CATEGORY_4,SUB_CATEGORY_5,SUB_CATEGORY_6,SUB_CATEGORY_7,SIZE
0,1111009477,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,15.0
1,1111009497,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,15.0


In [5]:
store_df.head(2)

,STORE_ID,ADDRESS_STATE_PROV_CODE_1,ADDRESS_STATE_PROV_CODE_2,ADDRESS_STATE_PROV_CODE_3,ADDRESS_STATE_PROV_CODE_4,MSA_CODE_1,MSA_CODE_2,MSA_CODE_3,MSA_CODE_4,MSA_CODE_5,MSA_CODE_6,MSA_CODE_7,MSA_CODE_8,MSA_CODE_9,SEG_VALUE_NAME,SALES_AREA_SIZE_NUM,AVG_WEEKLY_BASKETS
0,367,1,0,0,0,1,0,0,0,0,0,0,0,0,1,24721,12707
1,389,1,0,0,0,1,0,0,0,0,0,0,0,0,2,46073,24767


In [6]:
mega_df = train_df.merge(product_df, on = 'UPC', how = 'left')
mega_df = mega_df.merge(store_df, left_on = 'STORE_NUM', right_on = 'STORE_ID', how = 'left')
mega_df.head(2)

,WEEK_END_DATE,STORE_NUM,UPC,BASE_PRICE,FEATURE,DISPLAY,UNITS,MONTH,MANUFACTURER_1,MANUFACTURER_2,...,MSA_CODE_3,MSA_CODE_4,MSA_CODE_5,MSA_CODE_6,MSA_CODE_7,MSA_CODE_8,MSA_CODE_9,SEG_VALUE_NAME,SALES_AREA_SIZE_NUM,AVG_WEEKLY_BASKETS
0,2009-01-14,367,1111009477,1.57,0,0,13,1,1,0,...,0,0,0,0,0,0,0,1,24721,12707
1,2009-01-14,367,1111009497,1.39,0,0,20,1,1,0,...,0,0,0,0,0,0,0,1,24721,12707


In [7]:
mega_df.isnull().sum()

WEEK_END_DATE                0
STORE_NUM                    0
UPC                          0
BASE_PRICE                   0
FEATURE                      0
DISPLAY                      0
UNITS                        0
MONTH                        0
MANUFACTURER_1               0
MANUFACTURER_2               0
MANUFACTURER_3               0
MANUFACTURER_4               0
MANUFACTURER_5               0
MANUFACTURER_6               0
MANUFACTURER_7               0
MANUFACTURER_8               0
MANUFACTURER_9               0
CATEGORY_1                   0
CATEGORY_2                   0
CATEGORY_3                   0
CATEGORY_4                   0
SUB_CATEGORY_1               0
SUB_CATEGORY_2               0
SUB_CATEGORY_3               0
SUB_CATEGORY_4               0
SUB_CATEGORY_5               0
SUB_CATEGORY_6               0
SUB_CATEGORY_7               0
SIZE                         0
STORE_ID                     0
ADDRESS_STATE_PROV_CODE_1    0
ADDRESS_STATE_PROV_CODE_2    0
ADDRESS_

We will drop the 'STORE_ID' column because it's repeating.

In [8]:
mega_df.drop(columns = 'STORE_ID', axis = 1, inplace = True )

## 2. Creating a validation set

In [9]:
mega_df['WEEK_END_DATE'] = pd.to_datetime(mega_df['WEEK_END_DATE'])

In [10]:
weeks = mega_df['WEEK_END_DATE'].unique()

* We have stored the unique WEEK_END_DATE in **'weeks'** list so that it would be easier to split the data based on time. It will be used to create and train and validation split.

* We will keep one week gap between the train and validation set and train set will start from the very begining from where the data is available.

In [11]:
# Creating dictionaries that stores WEEK_END_DATES for training and validation sets (with 1 week gap)

def get_train_val_set(number = 1):
    
    validation_sets = []
    
    for n in range(number):
        x = {}
        
        x['validation_set'] = weeks[len(weeks)-n-1]
        x['train_set'] = weeks[len(weeks)-n-3]
        
        validation_sets.append(x)
        
    return validation_sets

We could go ahead with just 1 validation set to check the RMSLE score we are getting from different baseline models. However, taking multiple validation sets also allows us to look at the consistency of scores across multiple subsets of data.

Here, we will take 5 validation sets for starters

In [12]:
validation_sets = get_train_val_set(number=5)
validation_sets

[{'validation_set': numpy.datetime64('2011-09-28T00:00:00.000000000'),
  'train_set': numpy.datetime64('2011-09-14T00:00:00.000000000')},
 {'validation_set': numpy.datetime64('2011-09-21T00:00:00.000000000'),
  'train_set': numpy.datetime64('2011-09-07T00:00:00.000000000')},
 {'validation_set': numpy.datetime64('2011-09-14T00:00:00.000000000'),
  'train_set': numpy.datetime64('2011-08-31T00:00:00.000000000')},
 {'validation_set': numpy.datetime64('2011-09-07T00:00:00.000000000'),
  'train_set': numpy.datetime64('2011-08-24T00:00:00.000000000')},
 {'validation_set': numpy.datetime64('2011-08-31T00:00:00.000000000'),
  'train_set': numpy.datetime64('2011-08-17T00:00:00.000000000')}]

In [13]:
# Now, we will use that dictionary and store the train and validation sets as a list of tuples.

data_set = []

for data in validation_sets:
    
    training_data = mega_df[mega_df['WEEK_END_DATE'] <= data['train_set']]
    validation_data = mega_df[mega_df['WEEK_END_DATE'] == data['validation_set']]
    
    data_set.append((training_data, validation_data))

## 3. Creating Base Models
### (A) Mean Prediction

Now, we will create our first baseline model, MEAN PREDICTION. We will use the past data to take average on a group of STORE_NUM and UPC and use this to predict on the validaion set.

Evaluation Metric:  **Root Mean Squared Log Error**

In [14]:
# Function for RMSLE

def get_rmsle(true, predicted):
    return np.sqrt(msle(true, predicted))

In [15]:
train_rmsle = []
valid_rmsle = []

for i, data in enumerate(data_set):
    
    train, valid = data
    
    mean_prediction = train.groupby(['STORE_NUM', 'UPC'])['UNITS'].mean().reset_index()
    
    train = train.merge(mean_prediction, how = 'left', on = ['STORE_NUM', 'UPC'])
    valid = valid.merge(mean_prediction, how = 'left', on = ['STORE_NUM', 'UPC'])
    
    t_rmsle = get_rmsle(train['UNITS_x'], train['UNITS_y'])
    v_rmsle = get_rmsle(valid['UNITS_x'], valid['UNITS_y'])
    
    train_rmsle.append(t_rmsle)
    valid_rmsle.append(v_rmsle)
    
    print('RMSLE ON TRAINING SET: ',i+1, ': ', t_rmsle)
    print('RMSLE ON VALIDATION SET: ',i+1, ': ',v_rmsle)
    print('=====================================================================')
    
# get the mean RMSLE on train and validation set.     
print('Mean RMSLE on Train: ', np.mean(train_rmsle))
print('Mean RMSLE on Valid: ', np.mean(valid_rmsle))

RMSLE ON TRAINING SET:  1 :  0.590227758659263
RMSLE ON VALIDATION SET:  1 :  0.5887593934251679
RMSLE ON TRAINING SET:  2 :  0.5912322922347757
RMSLE ON VALIDATION SET:  2 :  0.626292442973742
RMSLE ON TRAINING SET:  3 :  0.591764652319905
RMSLE ON VALIDATION SET:  3 :  0.47834728926290576
RMSLE ON TRAINING SET:  4 :  0.591403566346982
RMSLE ON VALIDATION SET:  4 :  0.581157656189428
RMSLE ON TRAINING SET:  5 :  0.5916069602429402
RMSLE ON VALIDATION SET:  5 :  0.7181409175065901
Mean RMSLE on Train:  0.5912470459607733
Mean RMSLE on Valid:  0.5985395398715668


### (B). Simple Moving Average

* Now, we will use the Simple Moving Average, Like earlier we have used the average over the complete training period. Here, average will be taken on a specified period.
* We will use the predicted value as the average number of UNITS sold in last 8 weeks from a particular store of a particular product.
* As, we have one week gap between the train and validation set.

In [16]:
train_copy = train.copy()
valid_copy = valid.copy()
    
    # group the data by STORE_NUM and UPC and use rolling and mean function to calculate the moving average.
data_copy = train_copy.groupby(['STORE_NUM','UPC'])['UNITS_x'].rolling(2).mean().reset_index().set_index('level_2')

In [17]:
def get_sma(i, train, valid, no_of_weeks = 2):
    
    # Create a copy of train and validation datasets
    train_copy = train.copy()
    valid_copy = valid.copy()
    
    # Predicting the Moving average for specified number of weeks from train dataset, using rolling function
    data_copy = train_copy.groupby(['STORE_NUM', 'UPC'])['UNITS'].rolling(no_of_weeks).mean().reset_index().set_index('level_2')
    
    # Adding a moving average column to train_copy dataset 
    train_copy['moving_average'] = data_copy['UNITS']
    
    # the last prediction on train set will be used as prediction on validation set.
    # calculate the last_average dataframe by groupby using last function.
    last_average = train_copy.groupby(['STORE_NUM', 'UPC'])['moving_average'].last().reset_index()
    
    train_copy = train_copy[['WEEK_END_DATE', 'STORE_NUM', 'UPC', 'UNITS', 'moving_average']]
    valid_copy = valid_copy[['WEEK_END_DATE','STORE_NUM', 'UPC', 'UNITS']]
    
    # drop the null values in the dataframe
    train_copy.dropna(inplace=True)
    # merge the validation data with the last_average by left join
    valid_copy = valid_copy.merge(last_average, how= 'left', on= ['STORE_NUM', 'UPC'])
    
    # calculate the rmsle on train and validation data
    t_rmsle = get_rmsle(train_copy['UNITS'], train_copy['moving_average'])
    v_rmsle = get_rmsle(valid_copy['UNITS'], valid_copy['moving_average'])
    
    print('RMSLE ON TRAINING SET: ',i+1, ': ', t_rmsle)
    print('RMSLE ON VALIDATION SET: ',i+1, ': ',v_rmsle)
    print('=====================================================================')
    
    return t_rmsle, v_rmsle
    

In [18]:
train_rmsle_ma = []
valid_rmsle_ma = []

for i, data in enumerate(data_set):
    train, valid = data

    t_rmsle, v_rmsle = get_sma(i,train, valid, no_of_weeks=8)
    train_rmsle_ma.append(t_rmsle)
    valid_rmsle_ma.append(v_rmsle)
    
print('Mean RMSLE on Train: ', np.mean(train_rmsle_ma))
print('Mean RMSLE on Valid: ', np.mean(valid_rmsle_ma))    

RMSLE ON TRAINING SET:  1 :  0.5322812942885873
RMSLE ON VALIDATION SET:  1 :  0.5469206496913668
RMSLE ON TRAINING SET:  2 :  0.5332220629244886
RMSLE ON VALIDATION SET:  2 :  0.6421015703332319
RMSLE ON TRAINING SET:  3 :  0.5335488387739031
RMSLE ON VALIDATION SET:  3 :  0.46149085909723564
RMSLE ON TRAINING SET:  4 :  0.5324671481419397
RMSLE ON VALIDATION SET:  4 :  0.5878031103068386
RMSLE ON TRAINING SET:  5 :  0.5318550421893262
RMSLE ON VALIDATION SET:  5 :  0.7558881602487321
Mean RMSLE on Train:  0.532674877263649
Mean RMSLE on Valid:  0.598840869935481


### (C). Linear Regression

Now, we will try one Linear Regression Model and see how it performs on our dataset. We will use the same 5 validation sets and compare the results.
We will drop the columns like WEEK_END_DATE, STORE_NUM and UPC before training the model.

In [19]:
train_rmsle_lr = []
valid_rmsle_lr = []

for i, data in enumerate(data_set):
    
    train, valid = data
    
    train_x = train.drop(columns = ['STORE_NUM', 'WEEK_END_DATE', 'UPC', 'UNITS'])
    train_y = train['UNITS']
    
    valid_x = valid.drop(columns = ['STORE_NUM', 'WEEK_END_DATE', 'UPC', 'UNITS'])
    valid_y = valid['UNITS']
    
    LR = LinearRegression()
    LR.fit(train_x, train_y)
    
    train_predict = LR.predict(train_x).clip(min = 0)
    valid_predict = LR.predict(valid_x).clip(min = 0)
    
    train_rmsle = get_rmsle(train_y, train_predict)
    valid_rmsle = get_rmsle(valid_y, valid_predict)
    train_rmsle_lr.append(train_rmsle)
    valid_rmsle_lr.append(valid_rmsle)
    
    print('RMSLE ON TRAINING SET: ',i+1, ': ', train_rmsle)
    print('RMSLE ON VALIDATION SET: ',i+1, ': ',valid_rmsle)
    print('=====================================================================')
    

print('Mean RMSLE on Train: ', np.mean(train_rmsle_lr))
print('Mean RMSLE on Valid: ', np.mean(valid_rmsle_lr))    

RMSLE ON TRAINING SET:  1 :  0.9914831043039526
RMSLE ON VALIDATION SET:  1 :  0.9123950697836425
RMSLE ON TRAINING SET:  2 :  0.9923376653699222
RMSLE ON VALIDATION SET:  2 :  0.8961875754926829
RMSLE ON TRAINING SET:  3 :  0.9929938669223298
RMSLE ON VALIDATION SET:  3 :  0.9605326669030287
RMSLE ON TRAINING SET:  4 :  0.9934004111895244
RMSLE ON VALIDATION SET:  4 :  0.9563001362482264
RMSLE ON TRAINING SET:  5 :  0.9941397630796495
RMSLE ON VALIDATION SET:  5 :  0.9754139135309008
Mean RMSLE on Train:  0.9928709621730757
Mean RMSLE on Valid:  0.9401658723916964


We can see that Linear Regression has performed really bad. Even predicting the mean vaues would be better model. So, it is clear the target variable has no linear dependency on the avaiable features.

### (D). Decision Tree
Now, we will try one Tree Based Model. We will use the same 5 validation sets and compare the results.
We will drop the columns like WEEK_END_DATE, STORE_NUM and UPC before training the model.

In [20]:
train_rmsle_dt = []
valid_rmsle_dt = []

for i, data in enumerate(data_set):
    
    train, valid = data
    
    train_x = train.drop(columns = ['STORE_NUM', 'WEEK_END_DATE', 'UPC', 'UNITS'])
    train_y = train['UNITS']
    
    valid_x = valid.drop(columns = ['STORE_NUM', 'WEEK_END_DATE', 'UPC', 'UNITS'])
    valid_y = valid['UNITS']
    
    DT = DecisionTreeRegressor()
    DT.fit(train_x, train_y)
    
    train_predict = DT.predict(train_x).clip(min = 0)
    valid_predict = DT.predict(valid_x).clip(min = 0)
    
    train_rmsle = get_rmsle(train_y, train_predict)
    valid_rmsle = get_rmsle(valid_y, valid_predict)
    train_rmsle_dt.append(train_rmsle)
    valid_rmsle_dt.append(valid_rmsle)
    
    print('RMSLE ON TRAINING SET: ',i+1, ': ', train_rmsle)
    print('RMSLE ON VALIDATION SET: ',i+1, ': ',valid_rmsle)
    print('=====================================================================')
    

print('Mean RMSLE on Train: ', np.mean(train_rmsle_dt))
print('Mean RMSLE on Valid: ', np.mean(valid_rmsle_dt))    

RMSLE ON TRAINING SET:  1 :  0.30884041663509815
RMSLE ON VALIDATION SET:  1 :  0.49969133304590596
RMSLE ON TRAINING SET:  2 :  0.3088550257315775
RMSLE ON VALIDATION SET:  2 :  0.5547571043099567
RMSLE ON TRAINING SET:  3 :  0.30922263720867416
RMSLE ON VALIDATION SET:  3 :  0.5408993052181688
RMSLE ON TRAINING SET:  4 :  0.30870747395552295
RMSLE ON VALIDATION SET:  4 :  0.5704263401342783
RMSLE ON TRAINING SET:  5 :  0.3083229669013995
RMSLE ON VALIDATION SET:  5 :  0.6079641976345111
Mean RMSLE on Train:  0.30878970408645445
Mean RMSLE on Valid:  0.5547476560685641


So, we can see that Decision Tree performed way better than the LinearRegression and better than the Mean Prediction.

### (E). RandomForest
We just saw that the Decision Tree performed better than the Linear Regression Model. So, we will try one Ensemble Model of Decision Trees like RandomForest and compare the results on the same 5 validation sets.

In [21]:
train_rmsle_rf = []
valid_rmsle_rf = []

for i, data in enumerate(data_set):
    
    train, valid = data
    
    train_x = train.drop(columns = ['STORE_NUM', 'WEEK_END_DATE', 'UPC', 'UNITS'])
    train_y = train['UNITS']
    
    valid_x = valid.drop(columns = ['STORE_NUM', 'WEEK_END_DATE', 'UPC', 'UNITS'])
    valid_y = valid['UNITS']
    
    RF = RandomForestRegressor(random_state=0)
    RF.fit(train_x, train_y)
    
    train_predict = RF.predict(train_x).clip(min = 0)
    valid_predict = RF.predict(valid_x).clip(min = 0)
    
    train_rmsle = get_rmsle(train_y, train_predict)
    valid_rmsle = get_rmsle(valid_y, valid_predict)
    train_rmsle_rf.append(train_rmsle)
    valid_rmsle_rf.append(valid_rmsle)
    
    print('RMSLE ON TRAINING SET: ',i+1, ': ', train_rmsle)
    print('RMSLE ON VALIDATION SET: ',i+1, ': ',valid_rmsle)
    print('=====================================================================')
    

print('Mean RMSLE on Train: ', np.mean(train_rmsle_rf))
print('Mean RMSLE on Valid: ', np.mean(valid_rmsle_rf))    

RMSLE ON TRAINING SET:  1 :  0.3188464406065117
RMSLE ON VALIDATION SET:  1 :  0.532337664899423
RMSLE ON TRAINING SET:  2 :  0.3165633604463837
RMSLE ON VALIDATION SET:  2 :  0.5562040882768083
RMSLE ON TRAINING SET:  3 :  0.31383124125340356
RMSLE ON VALIDATION SET:  3 :  0.5502410047493668
RMSLE ON TRAINING SET:  4 :  0.31067513509906536
RMSLE ON VALIDATION SET:  4 :  0.5817354080038292
RMSLE ON TRAINING SET:  5 :  0.3083229669013995
RMSLE ON VALIDATION SET:  5 :  0.6079641976345111
Mean RMSLE on Train:  0.3136478288613528
Mean RMSLE on Valid:  0.5656964727127877


**Conclusions**
* We have seen that Tree Based Models have a better performance than other models.
* Although, RandomForest gives similar results to decision tree, its computational time is too much as compared to other models.
* We have a basic idea of what is the baseline.
* But still, we don't know what should be the right number of validation sets required and what should be the size of the training data.
